# This script should take in a list of search words and then save the last N tweets to a csv file after cleaning them


## Import utilities

In [ ]:
!pip install tweepy
import tweepy
import csv
import ssl
import time
import re
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
import numpy as np
import pandas as pd
fpath = "/content/drive/My Drive/Twitter analysis/"

Twitter login credentials

In [ ]:
# Add your Twitter API credentials
<hidden keys>

# Handling authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# Create a wrapper for the API provided by Twitter
api = tweepy.API(auth)

Obtain a list of tweets with provided search words

In [ ]:
keywords = [input ("Enter keywords of interest: ")]

N_tweets = 500
search_words = ""
for word in keywords:
  search_words+=word
  search_words+="  "
date_since = "2010-05-28"
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since,tweet_mode="extended").items(N_tweets)


Enter keywords of interest: Olympics


In [ ]:
filename = fpath+"tweets"
with open(filename+".csv", 'w') as file:
    global writer
    writer = csv.writer(file)
            # Add a header row to the CSV
    writer.writerow(["Tweet", "Matched Keywords", "Date", "User",
                        "Source", "Tweet ID", "Tweet URL"])
    for tweet_object in tweets:
      tweet = tweet_object.full_text

      '''Convert all named and numeric character references
            (e.g. &gt;, &#62;, &#x3e;) in the string s to the
            corresponding Unicode characters'''
      tweet = (tweet.replace('&amp;', '&').replace('&lt;', '<')
                 .replace('&gt;', '>').replace('&quot;', '"')
                 .replace('&#39;', "'").replace(';', " ")
                 .replace(r'\u', " ").replace(r'\https://'," "))
      
      #Remove https links from tweet
      tweet = re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '', tweet, flags=re.MULTILINE)

      # Save the keyword that matches the stream
      keyword_matches = []
      for word in keywords:
          if word.lower() in tweet.lower():
              keyword_matches.extend([word])

      keywords_strings = ", ".join(str(x) for x in keyword_matches)

            # Save other information from the tweet
      user = tweet_object.author.screen_name
      timeTweet = tweet_object.created_at
      source = tweet_object.source
      tweetId = tweet_object.id
      tweetUrl = "https://twitter.com/" + str(user)+ "/statuses/" + str(tweetId)

        # Exclude retweets, too many mentions and too many hashtags
      if not any((('RT @' in tweet, 'RT' in tweet,
                      tweet.count('@') >= 2, tweet.count('#') >= 3))):

        # Saves the tweet information in a new row of the CSV file
        writer.writerow([tweet, keywords_strings, timeTweet,
                                user, source, tweetId, tweetUrl])


In [ ]:
df = pd.read_csv(filename+".csv")
print("Most recent cleaned tweets scraped for '{}' -  \n".format(df['Matched Keywords'][0]))
df['Tweet'][:10]

Most recent cleaned tweets scraped for 'Olympics' -  



0    shall we sing "DYHTPS" reprise from Les Misera...
1    funny to think one day there will be no more o...
2    Lamda in Japan. Olympics in Japan. Malaysians ...
3    Canada picks Damian Warner as flag bearer for ...
4    You can’t sum all medals if you aggregate coun...
5    @India_AllSports All our Six individual medall...
6    Tokyo Games again show what the Olympics are a...
7    Tokyo Games again show what the Olympics are a...
8                              "controversial choice" 
9    Goodbye #Olympics! I enjoyed you throughly. I’...
Name: Tweet, dtype: object